# Defining Parameters and All Available Parameters

## Ways to define parameters

There are three ways to input parameters to the diagnostics:
1. Command line: For example: `set5_driver.py -p myparam.py --variables T` will set the variable to T.
2. Parameters file: In the command `set5_driver.py -p myparam.py`, the parameters file is `myparam.py`.
3. Diagnostics file: In the command `set5_driver.py -d mydiags.json`, the diagnostics file is `mydiags.json`.

**Each of these ways have a level of priority, with the command line input having the highest priority and the diagnostics file having the lowest priority.** 

### Examples

Say that we have the following files:

`myparams.py`:
```python
variables = 'T'
seasons = ['DJF', 'MAM', 'JJA', 'SON']
```

`mydiags.json`:
```json
{
    "set5": [
        {
            variables: 'PRECT',
            regions: ['global'],
            seasons: ['ANN']
        },
        {
            variables: 'SST',
            regions: ['ocean'],
            seasons: ['JJA']
        }
    ]
}
```

#### Running with just the parameters file

Running `set5_driver.py -p myparams.py` will just run the Set 5 diagnostics once with the parameters being: 
```python
variables = 'T'
seasons = ['DJF', 'MAM', 'JJA', 'SON']
```

#### Running with the diagnostics file

The `json` files support all of the non-programatic parameters covered below in the Available Parameters section. `json` files are also ways of defining multiple diagnostics runs, in `mydiags.json` defined above, we have two runs.

Running `set5_driver.py -d mydiags.json` will have two runs with the following parameters

Run 1:
```python
variables = 'PRECT'
regions = ['global']
seasons = ['ANN']
```
Run 2:
```python
variables = 'SST'
regions = ['ocean']
seasons = ['JJA']
```

Running `set5_driver.py -p myparams.py -d mydiags.json` will also have two runs, but the parameters in `myparams.py` will take priority over the ones described in `mydiags.json`. So the runs will be:

Run 1:
```python
variables = 'T'
regions = ['global']
seasons = ['DJF', 'MAM', 'JJA', 'SON']
```
Run 2:
```python
variables = 'T'
regions = ['ocean']
seasons = ['DJF', 'MAM', 'JJA', 'SON']
```

#### Running with command line arguments

With all of the three aforementioned ways of setting the parameters (just `myparams.py`, just `mydiags.json`, or both `myparams.py` and `mydiags.json`), command line arguments can be added to all. 

So running `set5_driver.py -p myparams.py -d mydiags.json --variables PREH2O` will have the variables in both runs be `PREH2O`:

Run 1:
```python
variables = 'PREH2O'
regions = ['global']
seasons = ['DJF', 'MAM', 'JJA', 'SON']
```
Run 2:
```python
variables = 'PREH2O'
regions = ['ocean']
seasons = ['DJF', 'MAM', 'JJA', 'SON']
```

## Available Parameters

The driver needs a parameters file to run. In these files, there is support for many features related to diagnostics. Below are all of the ones for Set 5.

Given a command like `set5_driver.py -p params.py`, the parameters in `params.py` will overwrite any predefined values for all of the runs.

### Parameters for diagnostics

The parameters in the section are ones related to test/reference specifications. Below are the parameters related to file I/O.

* `case_id`: the name of the folder where the results (nc files and plots) will be stored.
* `reference_data_path`: path to the reference (obs) data.
* `test_data_path`: path to the test (model) data.
* `reference_name`: the name of the reference (obs) file. This usually doesn't need to be defined, since you'll be running again either AMWG or your own diags. In the built-in parameter files for those, the reference_name is already defined. 
* `test_name`: the name of the test (model output) file.

The variables below are related to the actual climate-related functionality of the diagnostics.

* `variables`: What variables to use for this run. Note that currently, only single variable is supported: `variables="T"`.
* `season`: A list of season to use. Possible values are: "ANN", "DJF", "MAM", "JJA", "SON". Ex: `season=["ANN", "DJF", "MAM", "JJA", "SON"]`.
* `region`: A list of regions. See [this file](https://github.com/zshaheen/acme_diags/blob/master/acme_diags/derivations/default_regions.py) for a list of possible regions. Ex: `regions=["global","TROPICS"]`.
* `levels`: A list of pressure levels to use. Ex: `levels=[850.0, 200.0]`.
* `regrid_tool`: The regrid tool to use. **Possible values are: ____.** Is 'esmf' by default when no value is given.
* `regrid_method`: What regird method of the regrid tool to use. **Possible values are linear, conservative.** Is 'linear' by default when no value is given.

### Parameters for plotting

The figure below is an sample output. We use this to described what each plotting parameter does.

<img src=parameter_example.png alt="Figure1" style="width: 500px;"/>

Below are general plotting-related parameters. 

* `main_title`: Main title of the image. It's "PRECT ANN global" in the example and is 'Main Title' by default.
* `backend`: Can either be `vcs` or `cartopy`. Is `vcs` be default when no value is given.
* `output_file`: Name of the output file. Is `output` by default. Remember **not** to include an extension.
* `output_format`: A list of formats that yout want the plot to be output to. Can be something like `['png', 'pdf', 'svg']. `Is `['png']` when nothing is present.
* `canvas_size_w`: width of the image in pixels. Is 1212 by default.
* `canvas_size_h`: height of the image in pixels. Is 1628 by default.
* `arrows`: Is either True (default value) or False and will according show or hide the arrows on the legend for all of the graphs.
* `logo`: True (default value) to show the UV-CDAT logo on the vcs backend, False to not. Just keep it on please.

The parameters below are for each of the three plots (`test`, `reference`, and `diff`) in the image.

* `test_title`: the title for the test plot. It's "Test Title" in the image and is blank by default.
* `test_colormap`:  Wansn't define in the parameter, so is the default value **somecolormap**. A full list of colormaps is: ['AMIP', 'NCAR', 'bl_to_darkred', 'bl_to_drkorang', 'blends', 'blue2darkorange', 'blue2darkred', 'blue2green', 'blue2grey', 'blue2orange', 'blue2orange2red', 'blue_to_grey', 'blue_to_grn', 'blue_to_orange', 'blue_to_orgred', 'brown2blue', 'brown_to_blue', 'categorical', 'classic', 'default', 'green2magenta', 'grn_to_magenta', 'inferno', 'lightblue2darkblue', 'ltbl_to_drkbl', 'magma', 'plasma', 'rainbow', 'rainbow_no_grn', 'rainbownogreen', 'sequential', 'viridis', 'white2blue', 'white2green', 'white2magenta', 'white2red', 'white2yellow', 'white_to_blue', 'white_to_green', 'white_to_magenta', 'white_to_red', 'white_to_yellow']
* `contour_levels`: the levels on the legend of the test and reference plot. It's [0, 0.2, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 17] in the image but automatically gets the range by default.
* `test_units`: the units that are on the top-right of the test plot. It's "mm/day" in the image. If blank, it automatically gets the units from the test data. 

The `reference` and `diff` plots also have the same keywords which are semantically the same for their respective plots. Below are the values they hold for the image above.

* `reference_title`: "Reference title" in the image and is blank by default.
* `reference_colormap`: Wansn't define in the parameter, so is the default value **somecolormap**. 
* `contour_levels`: You only need one `contour_levels` in you script. It's used in the reference plot. [0, 0.2, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 17] in the image.
* `reference_units`: "mm/day" in the image. If blank, it automatically gets the units from the reference data. 

* `diff_title`: "Test - Reference" in the image. If blank, the default is "Model - Observation".
* `diff_colormap`: it "bl_to_darkred" in the image above and **this** by default. 
* `diff_levels`: [-6, -5, -4, -3, -2, -1, -0.5, 0, 0.5, 1, 2, 3, 4, 5, 6] in the image but automatically gets the range by default.
* `diff_units`: "mm/day" in the image. If blank, it automatically gets the units from the test - reference data. 